In [9]:
from math import sin, cos, sqrt, atan2, radians
from urllib.parse import quote
import json
import pandas as pd
import requests
import folium

API_KEY= 'gxy7IYotnmfdd1SrvXfy1hfG5z4HfU0pGbbVPclKWAy07uqNviLGHrnzp0J_Nt1hJ_vQd4AnIX3LQzh1SV9vybXQpR_MpvhhZTIUrx8b6K0Qp4yfU0wrPDQE_wDXXXYx'
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  

def request(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {'Authorization': 'Bearer %s' % api_key,}
    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()


def search(api_key, term, location):

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

def get_coordinates(search):
    url = 'https://nominatim.openstreetmap.org/search' 
    options = { 'q' : search, 'format' : 'json'}
    response = requests.get(url, params = options)            
    geodata = response.json()
    coords = { 'lat' : float(geodata[0]['lat']), 'lng' : float(geodata[0]['lon']) }
    return coords['lat'],coords['lng']

In [17]:
#businesses:{name:xx,location:{address1:xx,address2:xx},rating:xx,city:xx,}
while True:
    name=[]
    address=[]
    rating=[]
    location=[]
    
    get_type=input("Enter the type of the restaurant(Asian,French,Japanese...etc.): ")
    get_location=input("Enter a location: ")
    SEARCH_LIMIT = input("Enter the amount of restaurants you want to show on the map: ")
    restaurants=search(API_KEY,get_type,get_location)
    show_table = input("Do you want to show the table? (Y/N)")
    get_distance=input("Enter your location: ")
    user_location=get_coordinates(get_distance)
    
    try:
        for num in restaurants['businesses']:
            name.append(num['name'])
            address.append(num['location']['address1'])
            rating.append(num['rating'])
            location.append((num['coordinates']['latitude'],num['coordinates']['longitude']))
    except KeyError:
        print("Please enter something")
        break
        
    series_dict = { 'Name':  name,'Address': address,'Rating on Yelp': rating}  
    table=pd.DataFrame(series_dict)

    if show_table=='Y':
        display(table)

    coords = get_coordinates(get_location)
    show_map = folium.Map(location=coords, zoom_start=13)
    dictionary={'name':name,'rating':rating,'address':address,'location':location}
    for item in range(int(SEARCH_LIMIT)):
        loc=(dictionary['location'][item])
        message=f"Restaurant name:{dictionary['name'][item]} Rating:{dictionary['rating'][item]} Address:{dictionary['address'][item]}"
        marker = folium.Marker(location=loc,popup=message)
        show_map.add_child(marker)
    display(show_map)
    
    test_continue=input('Do you want to find another restaurant? Press "quit" to stop the program. ')
    if test_continue == 'quit':
        break
        

Enter the type of the restaurant(Asian,French,Japanese...etc.): chinese
Enter a location: syracuse
Enter the amount of restaurants you want to show on the map: 5
Do you want to show the table? (Y/N)Y
Enter your location: new york


,Name,Address,Rating on Yelp
0,Bamboo House Asian Restaurant,252 W Genesee St,4.5
1,Tang Flavor,413 S Warren St,4.0
2,Red Chili,2740 Erie Blvd E,4.0
3,Yang Di Chun BBQ,2042 Erie Blvd E,4.0
4,Old Chengdu Cafe,1113 E Fayette St,5.0


Do you want to find another restaurant? Press "quit" to stop the program. quit


In [24]:


R = 6373.0
lon1=radians(abs(user_location[1]))
lat1=radians(abs(user_location[0]))

for l in location:
    lon2=radians(abs(l[1]))
    lat2=radians(abs(l[0]))


    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    print('%.2f km'%distance)


    
    
        

315.19 km
314.51 km
312.70 km
313.20 km
313.90 km


In [ ]:
location